# 第 10 章 PCA 算法

---

In [1]:
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

np.set_printoptions(suppress=True)

A = np.array([[3, 2000],
              [2, 3000],
              [4, 5000],
              [5, 8000],
              [1, 2000]], dtype=float)

## 手写 PCA

In [2]:
# 得到每一列的均值
mean = A.mean(axis=0)
# 去均值以后的矩阵
norm = A - mean
# 去均值以后，再计算均值，就是 0 了
print(norm.mean(axis=0))
U, S, V = np.linalg.svd(norm.T.dot(norm))
print("奇异值:",S)

[0. 0.]
奇异值: [26000007.53846226        2.46153775]


In [7]:
U1, S1, V1 = np.linalg.svd(norm)
V1

array([[ 0.00053846,  0.99999986],
       [ 0.99999986, -0.00053846]])

In [3]:
U

array([[-0.00053846, -0.99999986],
       [-0.99999986,  0.00053846]])

In [4]:
S

array([26000007.53846226,        2.46153775])

In [5]:
V

array([[-0.00053846, -0.99999986],
       [-0.99999986,  0.00053846]])

In [27]:
# 取第 1 列
U_reduce = U[:, 0].reshape(-1, 1)
print("第 1 主成分:", U_reduce)
X_reduce = norm.dot(U_reduce)
X_reduce = X_reduce.reshape(-1, 1)

print("X_reduce:", X_reduce)

第 1 主成分: [[-0.00053846]
 [-0.99999986]]
X_reduce: [[ 1999.99971006]
 [ 1000.00039349]
 [-1000.00039349]
 [-4000.00049704]
 [ 2000.00078698]]


In [28]:
# 恢复的数据
X_recover = X_reduce.dot(U_reduce.T) + mean
print("X_recover:", X_recover)

X_recover: [[   1.92307713 2000.00057988]
 [   2.46153828 2999.99975148]
 [   3.53846172 5000.00024852]
 [   5.15384631 7999.99991716]
 [   1.92307655 1999.99950296]]


## 使用 scikit-learn 进行 PCA 降维

In [29]:
pca = PCA(n_components=1)

pca.fit(A)
X_reduce_sklearn = pca.transform(A)

X_recover_sklearn = pca.inverse_transform(X_reduce_sklearn)
print("X_recover_sklearn:", X_recover_sklearn)


X_recover_sklearn: [[   1.92307713 2000.00057988]
 [   2.46153828 2999.99975148]
 [   3.53846172 5000.00024852]
 [   5.15384631 7999.99991716]
 [   1.92307655 1999.99950296]]


In [32]:
pca.singular_values_**2

array([26000007.53846225])

In [37]:
pca.components_

array([[0.00053846, 0.99999986]])

In [38]:
pca.explained_variance_

array([6500001.88461556])

In [39]:
pca.explained_variance_ratio_

array([0.99999991])

In [33]:
dir(pca)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_fit',
 '_fit_full',
 '_fit_truncated',
 '_get_param_names',
 'components_',
 'copy',
 'explained_variance_',
 'explained_variance_ratio_',
 'fit',
 'fit_transform',
 'get_covariance',
 'get_params',
 'get_precision',
 'inverse_transform',
 'iterated_power',
 'mean_',
 'n_components',
 'n_components_',
 'n_features_',
 'n_samples_',
 'noise_variance_',
 'random_state',
 'score',
 'score_samples',
 'set_params',
 'singular_values_',
 'svd_solver',
 'tol',
 'transform',
 'whiten

In [7]:
import sklearn

print(sklearn.__version__)

0.19.2


一些帮助我回忆出 PCA 的数学原理的条目：

1、待定系数法，AU = B ，其中 A 是在原始坐标系下的坐标，横的一行表示一条数据，竖的一行表示某个特征的所有数据。U 是新坐标系中的单位向量，它的方向表示了坐标轴的方向。

其中 A 是 m 行 n 列，U 是 n 行 n 列，B 也是 m 行 n 列。

（1）B 的要求：B 的列向量之间线性无关（只剩下了线性无关的向量），并且方差按照从大到小排序，即 (1/(m-1)) BTB 是对角矩阵。
（2）最后我们发现，其实要找的 U 是这样的一个矩阵：使得(1/(m-1)) ATA 成为对角矩阵。

2、要求的是 U，把 U 砍掉一些列向量，得到 U_reduce，再和 A 相乘，就得到了降维以后的坐标。



## 特征归一化（这里没有标准化）

In [8]:
np.set_printoptions(suppress=True)

A = np.array([[3, 2000],
              [2, 3000],
              [4, 5000],
              [5, 8000],
              [1, 2000]], dtype=float)

mean = A.mean(axis=0)
norm = A - mean
scope = norm.max(axis=0) - norm.min(axis=0)
scalar = norm / scope

In [9]:
scalar

array([[ 0.        , -0.33333333],
       [-0.25      , -0.16666667],
       [ 0.25      ,  0.16666667],
       [ 0.5       ,  0.66666667],
       [-0.5       , -0.33333333]])

## 奇异值分解

In [10]:
U, S, V = np.linalg.svd(scalar.T.dot(scalar))
U

array([[-0.67710949, -0.73588229],
       [-0.73588229,  0.67710949]])

In [11]:
U_reduce = U[:, 0].reshape((2, 1))

## 降维

In [12]:
R = scalar.dot(U_reduce)
R 

array([[ 0.2452941 ],
       [ 0.29192442],
       [-0.29192442],
       [-0.82914294],
       [ 0.58384884]])

## 恢复

In [13]:
Z = np.dot(R, U_reduce.T)

In [14]:
Z

array([[-0.16609096, -0.18050758],
       [-0.19766479, -0.21482201],
       [ 0.19766479,  0.21482201],
       [ 0.56142055,  0.6101516 ],
       [-0.39532959, -0.42964402]])

## 进一步还原出原始的数据

In [15]:
scope

array([   4., 6000.])

In [16]:
A_recover = np.multiply(Z, scope) + mean

In [17]:
A_recover

array([[   2.33563616, 2916.95452015],
       [   2.20934082, 2711.06794139],
       [   3.79065918, 5288.93205861],
       [   5.2456822 , 7660.90959707],
       [   1.41868164, 1422.13588278]])

<br>
<br>

<br>
<br>

# scikit-learn 中使用 PCA

In [18]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

def std_PCA(**argv):
    scaler = MinMaxScaler()
    pca = PCA(**argv)
    pipeline = Pipeline([('scaler', scaler), ('pca', pca)])
    return pipeline


In [19]:
A

array([[   3., 2000.],
       [   2., 3000.],
       [   4., 5000.],
       [   5., 8000.],
       [   1., 2000.]])

In [20]:
pca = std_PCA(n_components=1)
pca.fit(A)
R2 = pca.transform(A)

In [21]:
R2

array([[-0.2452941 ],
       [-0.29192442],
       [ 0.29192442],
       [ 0.82914294],
       [-0.58384884]])

In [22]:
# inverse_transform 的操作就是 pipeline 反过来做的
A_recover2 = pca.inverse_transform(R2)
A_recover2

array([[   2.33563616, 2916.95452015],
       [   2.20934082, 2711.06794139],
       [   3.79065918, 5288.93205861],
       [   5.2456822 , 7660.90959707],
       [   1.41868164, 1422.13588278]])